# Ch35t


## Parser

In [2]:
# parser is used to parse a chest's json description
import parser

# if a Parser is instantiated with no input params, it
# will have an empty json
cp = parser.Parser()
print(cp.json())

None


## Manually load a chest
### From file

In [3]:
cp._load_file("examples/chest_01.json")
cp.json()

{'label': {'name': 'this is the name',
  'URI': 'this is the URI',
  'author': 'mala@sdf.org'},
 'hint': {'origin': 'this is the origin',
  'data': 'this is the hint data',
  'format': 'text/plain'},
 'payload': {'data': 'this is the payload data', 'format': 'text/plain'}}

### From URL

In [3]:
url = "http://3564020356.org/zelif/chest_01.json"
cp._load_url(url)
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': 'No method'}}

### From string

In [5]:
jj = '''
{
"hint": { "data": "this is the hint", "format": "text/plain" },
"payload": {"data": "this is the payload", 
  "format": "text/plain",
  "method": {"name": "this is the method"}
  }
}
'''
cp._load_string(jj)
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': {'name': 'this is the method'}}}

*NOTE* that manual load won't probably be required. If OCParser is initalised with a string that starts with `http://` or `https://` it will automatically call load_url, if it starts with `file://` it will automatically load a file, otherwise it will load a string. As we might want a URL to be considered as a string and not as a reference to some content, it is still possible to do that by instantiating an emtpy parser and loading the url as a string.

## Load automatically
Parser is able to automatically detect whether the JSON chest description is being passed as an URL, a file, or a string

In [3]:
cp = parser.Parser('{"hint": {"data": "This is a hint"}, "payload": {"data": "this is the payload"}}')
cp.json()

{'hint': {'data': 'This is a hint'},
 'payload': {'data': 'this is the payload'}}

In [6]:
cp = parser.Parser("http://3564020356.org/zelif/chest_01.json")
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': 'No method'}}

In [4]:
cp = parser.Parser("file://examples/chest_01.json")
cp.json()

{'label': {'name': 'this is the name',
  'URI': 'this is the URI',
  'author': 'mala@sdf.org'},
 'hint': {'origin': 'this is the origin',
  'data': 'this is the hint data',
  'format': 'text/plain'},
 'payload': {'data': 'this is the payload data', 'format': 'text/plain'}}

In [5]:
print(cp.hint())
print(cp.payload())

[i] Hint
    Origin: this is the origin
    Data: this is the hint data
    Format: text/plain

[i] Payload
    Origin: None
    Data: this is the payload data
    Format: text/plain
    Method: None



# Build hint object

Every hint is characterised by some `data` content and a `format`. Depending on the format, the hint will be interpreted in different ways (see e.g. https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/MIME_types/Common_types):

* `text/plain`: data can be simply printed
* `text/html`: html that needs to be interpreted/shown properly
* `image/*`: data will be displayed by the appropriate application
* `application/zip`: a zip file, to be unpacked in the chest directory
* `application/octet-stream`: this is a generic file (perhaps one wants to make the hint a riddle too?)

NOTE that:

* Data URLs (https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URLs) can be a good hint on how to embed data which is not just plain text, define mimetype, etc
* if we want to have BOTH online and offline stuff, we could have an `origin` for URL origins, `format` for their remote formats, `data` for the actual data that is downloaded from the URL and base64-encoded to allow for offline access.

SO:
* the fields under `hint` are `origin`, `format`, and `data`
* if we want to specify a hint by reference we provide `origin` (any URL) and `format` (the format of the data that will be returned by that URL, if the app has to parse it)
* if we want to specify a hint by value we provide `data` in a way that recalls data URLs, i.e. as in
```
data:[<mediatype>][;base64],<data>
```
* `mediatype` is specified as `format` and if `origin` and `data` are both present they share the same mediatype. Also format is, by default, `text/plain` (perhaps do `text/plain;charset=US-ASCII`?)
* `base64` is provided at the beginning of `data` optional if we want the data to be provided as encoded (great if we have binaries)
* `data` is the actual data. Differently from data URLs, it is NOT provided as URI-encoded

In [6]:
import hint as h

jj = '''{
"hint": { 
  "data": "this is the hint", 
  "format": "text/plain" }
}'''

cp._load_string(jj)
hint = h.Hint(cp.json())
print(hint.data)
print(hint.format)

### note that format is text/plain by default
jj = '''{
"hint": { 
  "data": "this is the hint"
}
}'''

cp._load_string(jj)
hint = h.Hint(cp.json())
print(hint.data)
print(hint.format)

this is the hint
text/plain
this is the hint
text/plain


In [7]:
jj = '''{
"hint": { 
  "origin": "http://3564020356.org/deserve.htm",
  "format": "text/html"
}
}'''

cp._load_string(jj)
print(cp.json())
hint = h.Hint(cp.json())
print(hint)

{'hint': {'origin': 'http://3564020356.org/deserve.htm', 'format': 'text/html'}}
[i] Hint
    Origin: http://3564020356.org/deserve.htm
    Data: None
    Format: text/html



# Build payload

In [9]:
import payload as p

jj = '''{
"payload": { 
  "data": "this is the payload"
}
}'''

cp._load_string(jj)
cp.json()
payload = p.Payload(cp.json())
print(payload.data)
print(payload.fmt)

this is the payload
text/plain


* TODO: add code for input- and output- formats

In [10]:
print(payload)

[i] Payload
    Origin: None
    Data: this is the payload
    Format: text/plain
    Method: None



In [12]:
cp = parser.Parser("file://examples/deserve.json")
cp.json()
print(cp.hint())
print(cp.payload())

[i] Hint
    Origin: http://3564020356.org/deserve.htm
    Data: MAL TIRRUEZF CR MAL RKZYIOL EX MAL OIY UAE RICF "MAL ACWALRM DYEUPLFWL CR ME DYEU MAIM UL IZL RKZZEKYFLF GH OHRMLZH"
    Format: text/plain

[i] Payload
    Origin: None
    Data: b535ffd9263f275c6747ec804fee198e
    Format: text/plain
    Method: md5



In [13]:
%load_ext autoreload
%autoreload 2

In [15]:
from ch35t import Chest

c = Chest("file://examples/deserve.json", chests_dir="./chests")
c.show_hint()

MAL TIRRUEZF CR MAL RKZYIOL EX MAL OIY UAE RICF "MAL ACWALRM DYEUPLFWL CR ME DYEU MAIM UL IZL RKZZEKYFLF GH OHRMLZH"


In [16]:
c.unlock()

Schweitser
Wrong key


In [17]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

In [18]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

In [19]:
pad = padding.PSS(
    mgf=padding.MGF1(hashes.SHA256()),
    salt_length=padding.PSS.MAX_LENGTH
)

In [20]:
message = b"A message I want to sign"

In [21]:
signature = private_key.sign(
    message,
    pad,
    hashes.SHA256()
)

In [22]:
public_key = private_key.public_key()

In [23]:
signature

b'FvN\xb5\xcb\xd7\xbe\x14\x13<^\x13\x99^`\xb1C\n\x88\xc5e\x83\x8b\xa1\x89\x8f\xa4\xf9\xdc*\xdd8\xfb7\xc3M\xc6\xed\rp$@(5\xd8\xa5\xb1\xe2A&\xa3\x81\xden\x08\x84\x8b\xe7\x12b:\xbfg\x02K\x9f\xde\x91{\xeb,\x8f\x10E\x121\xea\xebx\xee\xbb`\xec\xd9X\xfb\xdb\x80\xa3zPH\xbfNF\x85\x1a!H\x83\xdfB\xc0\x14\x1d(\xa3z\xca\xf7\'\xd6\x13z\xe4\xcaT\x03cz\x10t\xc1\x0ct\x8d~*d\'~\\\xdb\x88\x861z=+\x07K\x8c-\xdf\xad4k\xec\xbe\xc0B;\x13\x10\x89`P8\x06HAf9\t*\xbf\xa7\x89U\x02\'\xe0:\\\xda}\x0c\xc8R\x10\xc0,-I\x9a\x12d\xbbw\x1f\x91\xfeR*\x7f\xedwF\x10"JF[\x12V\xcf\x1ed\xb4\xf44\xfb\xa0\xddO\xbb\xe8\xdaJ\xadh6\x19\xb4\xbfL\xc8\xc3\x1b?\x8d\x90YWSNv\xc6\x14\x92V\xf6d$W\xd5\x8ac(\x01\xc4\xcc\x1df\xd02'

In [24]:
public_key.verify(
    signature,
    message,
    pad,
    hashes.SHA256()
)

In [6]:
import json

jj = '''
{
"hint": { "data": "this is the hint", "format": "text/plain" },
"payload": {"data": "this is the payload", 
  "method": {"name": "this is the method"},
  "format": "text/plain"
  }
}
'''
json.loads(jj)

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'method': {'name': 'this is the method'},
  'format': 'text/plain'}}

In [25]:
# import fastjsonschema
from jsonschema import validate, ValidationError
import json

In [28]:
with open("ch35t.schema.json", "rt") as f:
    s = json.load(f)
# print(schema)

with open("examples/deserve_b64.json", "rt") as f:
    riddle = json.load(f)

try:
    validate(schema=s, instance=riddle)
    print("Validation successful")
except ValidationError as e:
    print(e.message)

Validation successful
